#

In [11]:
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)
import tqdm
import backtrader as bt
import backtrader as bt
from datetime import datetime, timedelta
import json
import ccxt
import pprint

In [12]:
### API
with open('params.json') as f:
    params = json.load(f)

bitmex_api_key = params["apiKey"]    #Enter your own API-key here
bitmex_api_secret = params["apiSecret"] #Enter your own API-secret here

In [66]:
### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "3h": 240, "1d": 1440}
batch_size = 750
bitmex_client = bitmex(test=True, api_key=bitmex_api_key, api_secret=bitmex_api_secret)

/Users/jamescopperthwaite/opt/anaconda3/envs/myenv/lib/python3.5/site-packages/swagger_spec_validator/validator20.py:53: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  ref_dict['$ref'], '/'.join(path),


In [67]:

def get_all_bitmex(symbol, kline_size, start_datetime, end_datetime, save = False):
    filename = 'Fa1%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): 
        data_df = pd.read_csv(filename)
        return data_df
    else: 
        data_df = pd.DataFrame()
        
    delta_min = (end_datetime - start_datetime).total_seconds() / 60 # number of mins
    
    available_data = math.ceil(delta_min/binsizes[kline_size])
    
    
    rounds = math.ceil(available_data / batch_size)
    
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (start_datetime + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
            
    data_df["time"] = data_df["timestamp"].dt.time
    data_df["timestamp"] = data_df["timestamp"].astype(str).str[:-6]
    data_df = data_df.rename(columns={'timestamp': 'datetime', "vwap": "openinterest"})
    data_df.set_index('datetime', inplace=True)
    data_df = data_df.drop(["symbol", "trades"],axis=1)
    
    if save and rounds > 0: 
        data_df.to_csv(filename)
    print('All caught up..!')
    
    return data_df

In [68]:
start_datetime = datetime(2020, 6, 1, 0, 0, 0, 0)
end_datetime = datetime(2020, 6, 30, 23, 59, 59, 0)


bitmex_symbols = ["XBTUSD"]
datas = []
for symbol in bitmex_symbols:
    datas.append(get_all_bitmex(symbol, '1m', start_datetime, end_datetime, save = True))

/Users/jamescopperthwaite/opt/anaconda3/envs/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


HTTPBadRequest: 400 Bad Request: {'error': {'message': 'binSize is invalid.', 'name': 'HTTPError'}}

In [59]:
 broker_mapping = {
    'order_types': {
        bt.Order.Market: 'market',
        bt.Order.Limit: 'limit',
        bt.Order.Stop: 'stop',
        bt.Order.StopLimit: 'stop limit'
    },
    'mappings':{
        'closed_order':{
            'key': 'status',
            'value':'closed'
        },
        'canceled_order':{
            'key': 'result',
            'value':1}
    }
}

config = {'urls': {'api': 'https://testnet.bitmex.com'},
                   'apiKey': params["apiKey"],
                   'secret': params["apiSecret"],
                   'enableRateLimit': True,
                  }


In [59]:
exchange = ccxt.bitmex({
    'apiKey': params["apiKey"],
    'secret': params["apiSecret"],
})
if 'test' in exchange.urls:
    exchange.urls['api'] = exchange.urls['test'] 
pprint.pprint(exchange.fetch_balance())

{'BTC': {'free': 0.00745644,
         'total': 0.00793364,
         'used': 0.00047720000000000054},
 'free': {'BTC': 0.00745644},
 'info': [{'account': 319315,
           'action': '',
           'amount': 998918,
           'availableMargin': 745644,
           'commission': None,
           'confirmedDebit': 0,
           'currency': 'XBt',
           'excessMargin': 745644,
           'excessMarginPcnt': 0.1538,
           'grossComm': 121061,
           'grossExecCost': 0,
           'grossLastValue': 4849000,
           'grossMarkValue': 4849000,
           'grossOpenCost': 0,
           'grossOpenPremium': 0,
           'indicativeTax': 0,
           'initMargin': 0,
           'maintMargin': 47720,
           'marginBalance': 793364,
           'marginBalancePcnt': 0.1636,
           'marginLeverage': 6.111948613751065,
           'marginUsedPcnt': 0.0601,
           'pendingCredit': 0,
           'pendingDebit': 0,
           'prevRealisedPnl': -65270,
           'prevState': 

In [23]:
client = bitmex(test=True,
                api_key=params["apiKey"],
                api_secret=params["apiSecret"] )

In [60]:
ohlcv_candles = pd.DataFrame(client.Trade.Trade_getBucketed(
            binSize="1m",
            symbol='XBTUSD',
            count=300,
            reverse=True
        ).result()[0])
ohlcv_candles.head()

,close,foreignNotional,high,homeNotional,lastSize,low,open,symbol,timestamp,trades,turnover,volume,vwap
0,10301.5,500.0,10306.5,0.048535,498.0,10301.5,10308.5,XBTUSD,2020-09-11 08:57:00+00:00,3,4853492,500,10302.9054
1,10308.5,501.0,10308.5,0.048637,1.0,10301.0,10308.5,XBTUSD,2020-09-11 08:56:00+00:00,4,4863691,501,10301.8440
2,10308.5,1521.0,10317.5,0.147431,1.0,10302.0,10307.5,XBTUSD,2020-09-11 08:55:00+00:00,11,14743139,1521,10316.7234
3,10307.5,37853.0,10318.0,3.669269,50.0,10300.0,10309.0,XBTUSD,2020-09-11 08:54:00+00:00,16,366926865,37853,10316.7234
4,10309.0,813.0,10309.5,0.078918,1.0,10297.0,10297.5,XBTUSD,2020-09-11 08:53:00+00:00,9,7891848,813,10301.8440


In [61]:
upperband, middleband, lowerband = talib.BBANDS(ohlcv_candles.close.values[::-1], 
                timeperiod=100, nbdevup=1, 
                nbdevdn=1, matype=3)

print(ohlcv_candles.close.values[0])
print(middleband[-1])
print(upperband[-1])
print(lowerband[-1])


10301.5
10334.6509049
10384.4345013
10284.8673084


In [62]:
# 
client.Order.Order_new(symbol='XBTUSD', orderQty=100, price=10000).result()[0]

{'account': 319315,
 'avgPx': None,
 'clOrdID': '',
 'clOrdLinkID': '',
 'contingencyType': '',
 'cumQty': 0,
 'currency': 'USD',
 'displayQty': None,
 'exDestination': 'XBME',
 'execInst': '',
 'leavesQty': 100,
 'multiLegReportingType': 'SingleSecurity',
 'ordRejReason': '',
 'ordStatus': 'New',
 'ordType': 'Limit',
 'orderID': 'c4940e05-2731-3f3b-0c2f-b1bf40090451',
 'orderQty': 100,
 'pegOffsetValue': None,
 'pegPriceType': '',
 'price': 10000.0,
 'settlCurrency': 'XBt',
 'side': 'Buy',
 'simpleCumQty': None,
 'simpleLeavesQty': None,
 'simpleOrderQty': None,
 'stopPx': None,
 'symbol': 'XBTUSD',
 'text': 'Submitted via API.',
 'timeInForce': 'GoodTillCancel',
 'timestamp': datetime.datetime(2020, 9, 11, 8, 58, 11, 59000, tzinfo=tzutc()),
 'transactTime': datetime.datetime(2020, 9, 11, 8, 58, 11, 59000, tzinfo=tzutc()),
 'triggered': '',
 'workingIndicator': True}

In [63]:
client.Order.Order_cancelAll().result()

([{'account': 319315,
   'avgPx': None,
   'clOrdID': '',
   'clOrdLinkID': '',
   'contingencyType': '',
   'cumQty': 0,
   'currency': 'USD',
   'displayQty': None,
   'exDestination': 'XBME',
   'execInst': '',
   'leavesQty': 0,
   'multiLegReportingType': 'SingleSecurity',
   'ordRejReason': '',
   'ordStatus': 'Canceled',
   'ordType': 'Limit',
   'orderID': 'c4940e05-2731-3f3b-0c2f-b1bf40090451',
   'orderQty': 100,
   'pegOffsetValue': None,
   'pegPriceType': '',
   'price': 10000.0,
   'settlCurrency': 'XBt',
   'side': 'Buy',
   'simpleCumQty': None,
   'simpleLeavesQty': None,
   'simpleOrderQty': None,
   'stopPx': None,
   'symbol': 'XBTUSD',
   'text': 'Canceled: Canceled via API.\nSubmitted via API.',
   'timeInForce': 'GoodTillCancel',
   'timestamp': datetime.datetime(2020, 9, 11, 8, 58, 40, 798000, tzinfo=tzutc()),
   'transactTime': datetime.datetime(2020, 9, 11, 8, 58, 11, 59000, tzinfo=tzutc()),
   'triggered': '',
   'workingIndicator': False}],
 <bravado.request

In [ ]:
ohlcv_candles = pd.DataFrame(self.client.Trade.Trade_getBucketed(
            binSize=self.timeframe,
            symbol='XBTUSD',
            count=100,
            reverse=True
        ).result()[0])